# ThirdAI NeuralDB

>[NeuralDB](https://www.thirdai.com/neuraldb-enterprise/) 是由 [ThirdAI](https://www.thirdai.com/) 开发的、对 CPU 友好且可微调的向量存储库。

## 初始化

有两种初始化方法：
- 从零开始：基础模型
- 从检查点加载：加载先前保存的模型

对于以下所有初始化方法，如果设置了 `THIRDAI_KEY` 环境变量，则可以省略 `thirdai_key` 参数。

ThirdAI API 密钥可在 https://www.thirdai.com/try-bolt/ 获取。

您需要通过 `pip install -qU langchain-community` 安装 `langchain-community` 才能使用此集成。

In [ ]:
from langchain_community.vectorstores import NeuralDBVectorStore

# From scratch
vectorstore = NeuralDBVectorStore.from_scratch(thirdai_key="your-thirdai-key")

# From checkpoint
vectorstore = NeuralDBVectorStore.from_checkpoint(
    # Path to a NeuralDB checkpoint. For example, if you call
    # vectorstore.save("/path/to/checkpoint.ndb") in one script, then you can
    # call NeuralDBVectorStore.from_checkpoint("/path/to/checkpoint.ndb") in
    # another script to load the saved model.
    checkpoint="/path/to/checkpoint.ndb",
    thirdai_key="your-thirdai-key",
)

## 插入文档源

In [ ]:
vectorstore.insert(
    # If you have PDF, DOCX, or CSV files, you can directly pass the paths to the documents
    sources=["/path/to/doc.pdf", "/path/to/doc.docx", "/path/to/doc.csv"],
    # When True this means that the underlying model in the NeuralDB will
    # undergo unsupervised pretraining on the inserted files. Defaults to True.
    train=True,
    # Much faster insertion with a slight drop in performance. Defaults to True.
    fast_mode=True,
)

from thirdai import neural_db as ndb

vectorstore.insert(
    # If you have files in other formats, or prefer to configure how
    # your files are parsed, then you can pass in NeuralDB document objects
    # like this.
    sources=[
        ndb.PDF(
            "/path/to/doc.pdf",
            version="v2",
            chunk_size=100,
            metadata={"published": 2022},
        ),
        ndb.Unstructured("/path/to/deck.pptx"),
    ]
)

## 相似性搜索

要查询向量存储，您可以使用标准的 LangChain 向量存储方法 `similarity_search`，它会返回一个 LangChain Document 对象列表。每个 Document 对象代表索引文件中的一段文本。例如，它可能包含来自已索引 PDF 文件之一的段落。除了文本之外，该文档的 metadata 字段还包含有关文档 ID、文档来源（来自哪个文件）以及文档得分等信息。

In [ ]:
# This returns a list of LangChain Document objects
documents = vectorstore.similarity_search("query", k=10)

## 微调

NeuralDBVectorStore 可根据用户行为和领域特定知识进行微调。其微调方式有两种：
1. 关联：vectorstore 将一个源短语与一个目标短语关联起来。当 vectorstore 遇到源短语时，它也会考虑与目标短语相关的结果。
2. 提升权重：vectorstore 提升特定查询文档的分数。当您希望根据用户行为微调 vectorstore 时，此功能非常有用。例如，如果用户搜索“汽车是如何制造的”并且喜欢返回的 ID 为 52 的文档，那么我们可以提升 ID 为 52 的文档在“汽车是如何制造的”这个查询中的权重。

In [ ]:
vectorstore.associate(source="source phrase", target="target phrase")
vectorstore.associate_batch(
    [
        ("source phrase 1", "target phrase 1"),
        ("source phrase 2", "target phrase 2"),
    ]
)

vectorstore.upvote(query="how is a car manufactured", document_id=52)
vectorstore.upvote_batch(
    [
        ("query 1", 52),
        ("query 2", 20),
    ]
)